## creat by 思远

## disease-disease 

{C04;Neoplasms}                      {C17;Skin and Connective Tissue Diseases}  
     ↓                                                  ↓  
{C04.588;Neoplasms by Site}                  {C17.800;Skin Diseases}  
       ↘                                       ↓               ↘
          ↘                 {C17.800.090;Breast Diseases}      {a disease}
             ↘                     ↙   
        {C04.588.180;C17.800.090.500;Breast Neoplasms}  

paper: Inferring the human microRNA functional similarity and functional network based on microRNA-associated diseases

$DAG_{A}=(A,T_{A},E_{A})$  
a disease A, $T_{A}$ is the set of all ancestor nodes of A including node itself, $E_{A}$ is the set of corresponding links  
  
e.g. if A is Breast Neoplasms,   
$T_{A}$={Breast Neoplasms,Breast Diseases,Skin Diseases,Neoplasms by Site,Neoplasms,Skin and Connective Tissue Diseases},  
$E_{A}$={Neoplasms by Site->Breast Neoplasms,Neoplasms->Neoplasms by Site,Breast Diseases->Breast Neoplasms,Skin Diseases->Breast Diseases,Skin and Connective Tissue Diseases->Skin Diseases} 

$\left\{\begin{matrix}D_{A}(A)=1\\D_{A}(t)=max\{\Delta*D_{A}(t')|t'\in children\quad of\quad t\} &if & t\ne A \end{matrix}\right.$  
$DV(A)=\sum_{t\in T_{A}}D_{A}(t)$  
  
e.g. if A is Breast Neoplasms, set $\Delta$ is 0.5  
$DV(A)$=1.0 (Breast Neoplasms) + 0.5 (Breast Diseases) + 0.5 (Neoplasms by Site) + 0.5 × 0.5  
(Neoplasms) + 0.5 × 0.5 (Skin Diseases) + 0.5 × 0.5 × 0.5 (Skin and Connective Sissue Diseases)  
= 2.625

$S(A,B)=\frac{\sum_{t\in T_{A}\cap T_{B}}(D_{A}(t)+D_{B}(t))}{DV(A)+DV(B)}$  
  
e.g. if A is Breast Neoplasms , B is a disease  
$S(A,B)=\frac{0.5+0.5\times 0.5+0.5\times 0.5+0.5\times 0.5\times 0.5}{2.625+(1+0.5+0.5\times 0.5)}=0.257142857$

## lncRNA-lncRNA miRNA-miRNA

a disease group $DT=\{dt_{1},dt_{2},...,dt_{k}\}$  
similarity of $dt$ and $DT$ $S(dt,DT)=\max_{1\le i \le k}(S(dt,dt_{i}))$  
two lncRNAs(miRNAs) $A,B$ consider all diseases $DT_{1},DT_{2}$  
$sim(A,B)=\frac{\sum_{1\le i\le m}S(dt_{1i},DT_{2})+\sum_{1\le j \le n}S(dt_{2j},DT_{1})}{m+n}$  
if m=n=0 then $sim(A,B)=0$

e.g. lncRNA(miRNA) A,B  
$dd=\left(\begin{matrix}1&0.1&0.2\\0.1&1&0.3\\0.2&0.3&1\end{matrix}\right)$
$ld=\left(\begin{matrix}1&1&0\\0&1&1\end{matrix}\right)$  
$DT_{1}=\{dt_{11}=d_{1},dt_{12}=d_{2}\},DT_{2}=\{dt_{21}=d_{2},dt_{22}=d_{3}\},m=n=2$  
$\sum_{1\le i\le m}S(dt_{1i},DT_{2})=S(d_{1},DT_{2})+S(d_{2},DT_{2})=max(S(d_{1},d_{2}),S(d_{1},d_{3}))+max(S(d_{2},d_{2})+S(d_{2},d_{3}))=1.2$  
$\sum_{1\le j\le n}S(dt_{2j},DT_{1})=S(d_{2},DT_{1})+S(d_{3},DT_{1})=max(S(d_{2},d_{1})+S(d_{2},d_{2}))+max(S(d_{3},d_{1})+S(d_{3},d_{2}))=1.3$  
$sim(A,B)=0.625$

In [ ]:
def calculate_sim(ld,dd):
    s1=ld.shape[0]
    ll=torch.eye(s1)
    m2=dd*ld[:,None,:]
    m1=ld[:,:,None]
    for x,y in itertools.permutations(torch.linspace(0,s1-1,s1,dtype=torch.long),2):
        x,y=x.item(),y.item()
        m=m1[x,:,:]*m2[y,:,:]
        if ld[x].sum()+ld[y].sum()==0:
            ll[x,y]=0
        else:
            ll[x,y]=(m.max(dim=0,keepdim=True)[0].sum()+m.max(dim=1,keepdim=True)[0].sum())/(ld[x].sum()+ld[y].sum())
    return ll